In [2]:
import pandas as pd
from datasets import load_dataset
import json
from sentence_transformers import SentenceTransformer, util
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, TaskType, PeftModel
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
import torch
from datasets import Dataset
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from trl import SFTTrainer, SFTConfig
callbacks = EarlyStoppingCallback(early_stopping_patience=3,
                                 early_stopping_threshold=0.01)

ImportError: /home/shegun93/anaconda3/envs/n_project/lib/python3.9/site-packages/flash_attn_2_cuda.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs

In [2]:
#=============================
# 1. Loading fine-tuned model
#==========================

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "./nairs-2e"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)
use_quantization_config = True 
attn_implementation = "flash_attention_2" if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8) else "sdpa"

print(f"[INFO] Using attention implementation: {attn_implementation}")
print(f"[INFO] Using model_id: {model_id}")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
nairs = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
                                             torch_dtype = torch.float16,
                                              quantization_config=quantization_config if use_quantization_config else None,
                                               low_cpu_mem_usage=True,
                                                 device_map = "auto",
                                                attn_implementation=attn_implementation
                                           )
if not use_quantization_config:
    nairs.to("cuda")

[INFO] Using attention implementation: flash_attention_2
[INFO] Using model_id: ./nairs-2e


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
#=========================
#2. Zero shot
#=========================
prompt = "Yesterday we were thought in class the principle of relativity however, I do not quiet understand it"
input_ids = tokenizer(prompt, return_tensors = "pt").to("cuda")

In [6]:
outputs = nairs.generate(
    **input_ids,
    max_new_tokens = 300,
    temperature = 0.7,
    do_sample = True
)
out = tokenizer.decode(outputs[0], skip_special_token=True)
print("nairs generated:", out)

nairs generated: <s> Yesterday we were thought in class the principle of relativity however, I do not quiet understand it.

Question: What is the effect of relative motion on time?

Options:
A: Time appears to decrease
B: Time appears to increase
C: Time appears to remain the same
D: Time appears to stop

Answer: A: Time appears to decrease

Explanation: According to relativity of time, time appears to decrease for an observer moving at high speeds relative to an observer.

Options:
A: Time appears to decrease
B: Time appears to increase
C: Time appears to remain the same
D: Time appears to stop

Answer: A: Time appears to decrease

Explanation: According to relativity of time, time appears to decrease for an observer moving at high speeds relative to an observer.

Options:
A: Time appears to decrease

Explanation: According to relativity of time, time appears to decrease for an observer moving at high speeds relative to an observer.

Answer: A: Time appears to decrease

Explanation: A

In [7]:
#=========================
#2. One shot
#=========================

In [8]:
def generate_physics_assessment(nairs, tokenizer, context, max_new_tokens=300, temperature=0.8):
    """
    Generates properly formatted physics assessments with guaranteed structure.
    Implements multiple fallback mechanisms for reliable output.
    """
    # 1. Create an explicit few-shot prompt with clear formatting examples
    prompt = f"""Generate an assessment question with options and provide a detailed explanation using EXACTLY this format:

Example 1:
Context: When soldiers march across a suspension bridge...
Question: Why are marching soldiers advised to break step on bridges?
Options:
A: To reduce air resistance
B: To prevent resonance
C: To minimize friction
D: To decrease bridge weight
Answer: B
Explanation: Marching soldiers are advised to break step on bridges to prevent resonance. When soldiers march in unison, their rhythmic footsteps can match the bridge's natural frequency. This matching of frequencies can cause the bridge to oscillate with increasing amplitude, potentially leading to structural damage. Breaking step ensures that the periodic force isn't applied at the bridge's natural frequency, preventing dangerous resonance effects.

Now generate for:
Context: {context}
Question:"""

    # 2. Generate the output with conservative parameters
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = nairs.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1
    )
    
    # 3. Extract and clean the generated text
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_part = full_output.split("Question:")[-1].strip()
    return generated_part

In [9]:
context = "Yesterday we were thought in class the principle of relativity however, I do not quiet understand it"
problem = generate_physics_assessment(nairs, tokenizer, context)
print(f"Nairs Generation: {problem}")

Nairs Generation: What principle is NOT related to motion?
Options:
A: Newton's laws
B: Pascal's principle
C: Bernoulli's theorem
D: Maxwell's equations
Answer: B
Explanation: The principle of relativity states that relative motion causes time and space changes. It does not relate to Newton's laws of motion, which describe how objects move under various forces without considering relative motion.

Please answer with the format provided.


In [ ]:
#===========================
# 3. Merging and unloading
#===========================

In [ ]:
fine_tuned_path = "./nairs-2e"
model = PeftModel.from_pretrained(, fine_tuned_path)
merged_model = model.merge_and_unload()

In [ ]:
merged_model.save_pretrained("Research")
tokenizer.save_pretrained("Research")